In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.0 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.1 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import LongformerModel, LongformerTokenizer
from transformers import RobertaTokenizer, RobertaModel
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from transformers import DistilBertModel, DistilBertTokenizerFast
import tensorflow as tf
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from transformers import EarlyStoppingCallback
from torch.optim import AdamW
from transformers import AutoModel
from transformers import AlbertModel
from transformers import AlbertTokenizer
from transformers import XLNetModel
from transformers import XLNetTokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Thesis/commentbig5sample.csv", encoding='latin-1')

In [ ]:

# Drop columns
df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)

In [ ]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

# Preprocess tweets: tokenize and pad/truncate
max_length = 64  # Choose the max_length that best suits your data

df['body'] = df['body'].apply(lambda x: tokenizer.encode_plus(x, truncation=True, padding='max_length', max_length=max_length))



# Split into train and test first
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

# Split df_train into train and validation
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=42)


In [ ]:
# Define a PyTorch dataset
class PersonalityDataset(Dataset):
    def __init__(self, tweets, targets):
        self.tweets = tweets
        self.targets = targets

    def __len__(self):
        return len(self.tweets)

    def __getitem__(self, idx):
      input_ids = torch.tensor(self.tweets[idx]['input_ids'], dtype=torch.long).squeeze(0)
      attention_mask = torch.tensor(self.tweets[idx]['attention_mask'], dtype=torch.long).squeeze(0)
      targets = torch.tensor(self.targets[idx], dtype=torch.float)

      return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'targets': targets
      }

In [ ]:

class AlbertForPersonalityTraits(torch.nn.Module):
    def __init__(self):
        super(AlbertForPersonalityTraits, self).__init__()
        self.albert = AlbertModel.from_pretrained('albert-base-v2')
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 5)  # Update the input size to match the ALBERT hidden size

    def forward(self, input_ids, attention_mask):
        outputs = self.albert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        output = self.dropout(pooled_output)
        output = self.linear(output)
        return output

In [ ]:
# Prepare data loaders
batch_size = 16
train_dataset = PersonalityDataset(df_train['body'].tolist(), df_train[['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']].values)
val_dataset = PersonalityDataset(df_val['body'].tolist(), df_val[['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']].values)
test_dataset = PersonalityDataset(df_test['body'].tolist(), df_test[['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Instantiate the model and define optimizer and loss function

model = AlbertForPersonalityTraits()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.MSELoss()

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.dense.bias', 'predictions.dense.weight', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.weight', 'predictions.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Training loop
epochs = 10

# Early stopping parameters
patience = 3
best_val_loss = float('inf')
patience_counter = 0
accumulation_steps = 16

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for i, batch in enumerate(train_loader):
        optimizer.zero_grad() if i % accumulation_steps == 0 else None

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['targets'].to(device)

# Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, targets)

        # Backward pass
        loss = loss / accumulation_steps
        loss.backward()
        if (i+1) % accumulation_steps == 0 or i+1 == len(train_loader):  # update on last step even if accumulation steps have not been reached
            optimizer.step()
            optimizer.zero_grad()  # make sure to reset gradient after updating

        total_loss += loss.item()

        # Delete unnecessary tensors to save GPU memory
        del input_ids
        del attention_mask
        del targets
        del outputs
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Train loss at epoch {epoch + 1}: {avg_train_loss}")

    # Validation loop
    model.eval()
    val_total_loss = 0
    all_predictions = [[] for _ in range(5)]  # Separate predictions for each trait
    all_targets = [[] for _ in range(5)]  # Separate targets for each trait

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['targets'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs, targets)

            val_total_loss += loss.item()

            # Split predictions and targets for each trait
            for i in range(5):
                all_predictions[i].extend(outputs[:, i].cpu().numpy())
                all_targets[i].extend(targets[:, i].cpu().numpy())

    avg_val_loss = val_total_loss / len(val_loader)
    print(f"Validation loss at epoch {epoch + 1}: {avg_val_loss}")

    # Check if early stopping conditions are met
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0  # reset counter
        # save the best model
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1  # increment counter
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break  # break out of the training loop


# After the validation loop...
trait_names = ["Extroversion", "Neuroticism", "Agreeableness", "Conscientiousness", "Openness"]

for i in range(5):
    predictions = all_predictions[i]
    targets = all_targets[i]

    mse = mean_squared_error(targets, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(targets, predictions)

    print(f"For {trait_names[i]}:")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"R^2: {r2}\n")


Train loss at epoch 1: 0.020949351601302624
Validation loss at epoch 1: 0.15420150650399073
Train loss at epoch 2: 0.012400874224576083
Validation loss at epoch 2: 0.13274947926402092
Train loss at epoch 3: 0.010650086072696881
Validation loss at epoch 3: 0.11511270595448357
Train loss at epoch 4: 0.01040455424650149
Validation loss at epoch 4: 0.12429942616394588
Train loss at epoch 5: 0.009944777588614008
Validation loss at epoch 5: 0.14309840223618916
Train loss at epoch 6: 0.012199122890491377
Validation loss at epoch 6: 0.20328156862940108
Early stopping triggered.
For Extroversion:
MSE: 0.10906654596328735
RMSE: 0.3302522599697113
R^2: -0.039414756559123854

For Neuroticism:
MSE: 0.21643109619617462
RMSE: 0.4652215540409088
R^2: -1.6072557156019829

For Agreeableness:
MSE: 0.2309194803237915
RMSE: 0.48054081201553345
R^2: -1.5645423432701397

For Conscientiousness:
MSE: 0.2199145257472992
RMSE: 0.46895045042037964
R^2: -1.288409180285346

For Openness:
MSE: 0.23444019258022308
RM

In [ ]:
# Evaluation on test set
model.eval()
test_total_loss = 0
all_predictions = [[] for _ in range(5)]  # Separate predictions for each trait
all_targets = [[] for _ in range(5)]  # Separate targets for each trait

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['targets'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, targets)

        test_total_loss += loss.item()

        # Split predictions and targets for each trait
        for i in range(5):
            all_predictions[i].extend(outputs[:, i].cpu().numpy())
            all_targets[i].extend(targets[:, i].cpu().numpy())

trait_names = ["Ext", "Neu", "Agre", "Con", "Ope"]

for i in range(5):
    predictions = all_predictions[i]
    targets = all_targets[i]

    mse = mean_squared_error(targets, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(targets, predictions)
    r2 = r2_score(targets, predictions)

    print(f"For {trait_names[i]}:")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"R^2: {r2}\n")

For Ext:
MSE: 0.09623270481824875
RMSE: 0.31021395325660706
MAE: 0.2727999687194824
R^2: -0.010668683424801984

For Neu:
MSE: 0.20601071417331696
RMSE: 0.4538840353488922
MAE: 0.3827022910118103
R^2: -1.6285200481493542

For Agre:
MSE: 0.22969263792037964
RMSE: 0.4792625904083252
MAE: 0.3958872854709625
R^2: -1.3325824031027502

For Con:
MSE: 0.21826113760471344
RMSE: 0.46718427538871765
MAE: 0.37974071502685547
R^2: -1.3819306492311179

For Ope:
MSE: 0.22978565096855164
RMSE: 0.47935962677001953
MAE: 0.3998108506202698
R^2: -1.2875289122155502



In [ ]:
df_predictions = pd.DataFrame(np.transpose(all_predictions), columns=trait_names)


correlation_matrix_predictions = df_predictions.corr()
print("\nCorrelation matrix for predicted values:")
print(correlation_matrix_predictions)


Correlation matrix for predicted values:
           Ext       Neu      Agre       Con       Ope
Ext   1.000000 -0.102184 -0.012400 -0.018411  0.005190
Neu  -0.102184  1.000000  0.954930  0.980770  0.951627
Agre -0.012400  0.954930  1.000000  0.962814  0.965302
Con  -0.018411  0.980770  0.962814  1.000000  0.956742
Ope   0.005190  0.951627  0.965302  0.956742  1.000000


In [ ]:
def color_cells(val):
    if val < 0:
        return 'background-color: red'
    elif val == 1:
        return 'background-color: yellow'
    else:
        return 'background-color: green'

styled_df = correlation_matrix_predictions.style.applymap(color_cells)

styled_df